## Import the relevant packages

In [2]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [4]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True: provides us with a tuple-structure dataset for inputs and targets.

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']


num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    # dividing each element by 255 to get the desired result -> all elements will be between 0 and 1 
    return image, label


scaled_train_and_validation_data = mnist_train.map(scale)


test_data = mnist_test.map(scale)


# shuffling the data
BUFFER_SIZE = 10000
# while dealing with enormous datasets, we can't shuffle the whole dataset in one go
# so instead TF only stores BUFFER_SIZE samples in memory at a time and shuffles them

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)


validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100
# while training data, it would be easier to iterate over the different batches

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data)) # takes next batch (it is the only batch)

D:\data science-anaconda\envs\py3-TF2.0\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|                                                                         | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|                                                                         | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|                                                                         | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|                                                                         | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|                                                                              | 0/9 [00:06<?, ? MiB/s]

Dl Size...:   0%|                                                                              | 0/9 [00:06<?, ? MiB/s]

Dl Size...:   0%|                                         

Dataset mnist downloaded and prepared to C:\Users\User\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


## Model

### Outline the model

In [16]:
input_size = 784
output_size = 10
hidden_layer_size = 50


model = tf.keras.Sequential([
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # Flatten: allows us to create a feed forward neural network
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer

    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

### Choose the optimizer and the loss function

In [18]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [20]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
540/540 - 2s - loss: 0.4067 - accuracy: 0.8829 - val_loss: 0.2036 - val_accuracy: 0.9373 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1804 - accuracy: 0.9469 - val_loss: 0.1503 - val_accuracy: 0.9560 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.1370 - accuracy: 0.9596 - val_loss: 0.1224 - val_accuracy: 0.9653 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.1108 - accuracy: 0.9667 - val_loss: 0.1236 - val_accuracy: 0.9645 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0937 - accuracy: 0.9716 - val_loss: 0.0969 - val_accuracy: 0.9720 - 2s/epoch - 3ms/step


## Test the model

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

      1/Unknown - 1s 1s/step - loss: 0.0897 - accuracy: 0.97 - 1s 1s/step - loss: 0.0897 - accuracy: 0.9728